In [2]:
# -*- coding: utf-8 -*-
"""
Created on Sat Jan 18 15:28:30 2020

@author: Yuan
"""

import pandas as pd
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk

# =============================================================================
# nltk.download('wordnet')
# =============================================================================
data = pd.read_csv('g.csv', error_bad_lines=False, encoding = "ISO-8859-1")
data_text = data[['title','simple description','description for Gccampus']]
documents = data_text

documents['title'] = documents['title'].fillna('999')
documents = documents.drop(documents[(documents.title=='999')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cours de EFPC / CSPS courses')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cours provenant d\'une autre ministère / Other Government Department courses')].index.tolist())
documents = documents.drop(documents[(documents.title=='Externals Ressources Links')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cours de Skillsoft courses')].index.tolist())
documents = documents.drop(documents[(documents.title=='Delegation of Authority - video')].index.tolist())
documents = documents.drop(documents[(documents.title=='Blogs')].index.tolist())
documents = documents.drop(documents[(documents.title=='Cases Studies')].index.tolist())
documents = documents.drop(documents[(documents.title=='hq')].index.tolist())
documents = documents.drop(documents[(documents.title=='Video List')].index.tolist())

for i in documents.index:
    if(' '.join(str(documents['simple description'][i]).split()[:10])==' '.join(str(documents['description for Gccampus'][i]).split()[:10])):
        documents['simple description'][i]= ""

documents.to_csv("clean.csv")

documents['combine'] = documents[['title','simple description','description for Gccampus']].apply(lambda x : '{} {} {}'.format(x[0],x[1],x[2]), axis=1)

# documents = documents['combine']

documents.to_csv("combine.csv")

# df.drop('name', axis=1)

stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result


doc_sample = documents[documents.index == 4].values[0][0]
print('\noriginal document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\ntokenized and lemmatized document: ')
print(preprocess(doc_sample))
processed_docs = documents['combine'].map(preprocess)
print(documents['combine'].shape[0])
processed_docs = processed_docs[:documents['combine'].shape[0]]

# Bag of words on the dataset
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

import pyLDAvis.gensim
import pyLDAvis
pyLDAvis.enable_notebook()

lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=20, id2word=dictionary, passes=2, workers=3)


from gensim.test.utils import datapath

lda_model.save("word2vec.model")

lda_model = gensim.models.ldamodel.LdaModel.load("word2vec.model")

pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

# for idx, topic in lda_model.print_topics(-1):
#     print('Topic: {} \nWords: {}'.format(idx, topic))


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



original document: 
['Being', 'Brief:', 'Making', 'a', 'Request', 'Efficiently', '(B196)']

tokenized and lemmatized document: 
['brief', 'make', 'request', 'effici']
1117
0 activ
1 appli
2 asset
3 awar
4 best
5 contribut
6 cours
7 department
8 direct
9 foundat
10 gain


//anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.032017  0.017132       1        1  9.827771
15     0.016562 -0.031535       2        1  7.816177
10    -0.008034 -0.063154       3        1  6.924240
12     0.061171  0.044348       4        1  6.587027
0     -0.029769 -0.094509       5        1  6.380157
13     0.061329  0.075583       6        1  6.243073
17     0.041320 -0.041110       7        1  6.019348
18     0.013036 -0.056708       8        1  5.980678
4      0.068038 -0.050993       9        1  5.688814
14     0.017780 -0.046836      10        1  5.334339
6     -0.027673 -0.028997      11        1  5.231436
3     -0.118230  0.013050      12        1  4.585053
8      0.039861  0.027018      13        1  4.131954
16     0.041022  0.046179      14        1  3.976137
11    -0.054087  0.018354      15        1  3.665724
2      0.057919 -0.057648      16        1  3.173920
1     -0.104012  0.024350      17        1  2.498256
19    -0.041297 -0.005325      18        1  2.092387
9      0.051849  0.149440      19        1  1.996276
5     -0.118803  0.061361      20        1  1.847234, topic_info=     Category        Freq       Term       Total  loglift  logprob
789   Default  165.000000      gcdoc  165.000000  30.0000  30.0000
15    Default  670.000000      manag  670.000000  29.0000  29.0000
121   Default  186.000000      chang  186.000000  28.0000  28.0000
11    Default  370.000000     govern  370.000000  27.0000  27.0000
29    Default  397.000000     servic  397.000000  26.0000  26.0000
400   Default   68.000000     author   68.000000  25.0000  25.0000
402   Default   62.000000      deleg   62.000000  24.0000  24.0000
787   Default   84.000000   document   84.000000  23.0000  23.0000
1151  Default   88.000000     mental   88.000000  22.0000  22.0000
13    Default  208.000000     inform  208.000000  21.0000  21.0000
27    Default  121.000000      secur  121.000000  20.0000  20.0000
556   Default   50.000000     negoti   50.000000  19.0000  19.0000
1418  Default   52.000000     custom   52.000000  18.0000  18.0000
25    Default  348.000000     public  348.000000  17.0000  17.0000
150   Default  296.000000       work  296.000000  16.0000  16.0000
380   Default   98.000000    financi   98.000000  15.0000  15.0000
122   Default  170.000000   communic  170.000000  14.0000  14.0000
181   Default  215.000000     polici  215.000000  13.0000  13.0000
274   Default   38.000000      ethic   38.000000  12.0000  12.0000
224   Default  127.000000       busi  127.000000  11.0000  11.0000
56    Default  294.000000     canada  294.000000  10.0000  10.0000
744   Default   69.000000      innov   69.000000   9.0000   9.0000
156   Default   95.000000    languag   95.000000   8.0000   8.0000
416   Default   68.000000      train   68.000000   7.0000   7.0000
378   Default   75.000000     cultur   75.000000   6.0000   6.0000
639   Default   53.000000  difficult   53.000000   5.0000   5.0000
811   Default   26.000000    version   26.000000   4.0000   4.0000
683   Default   83.000000     deputi   83.000000   3.0000   3.0000
155   Default   36.000000   influenc   36.000000   2.0000   2.0000
352   Default  177.000000   workplac  177.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
1917  Topic20    1.670726     bureau    7.275951   2.5202  -5.9115
779   Topic20    3.409389      start   23.748680   2.0505  -5.1983
295   Topic20    5.759855       self   66.142426   1.5506  -4.6739
17    Topic20    5.564274      onlin   66.748383   1.5069  -4.7084
353   Topic20    2.132939   accommod   12.838076   2.1966  -5.6673
209   Topic20    4.994132     procur   70.006912   1.3512  -4.8165
129   Topic20    7.394484    employe  177.311295   0.8143  -4.4241
559   Topic20    3.628544    control   41.739006   1.5489  -5.1360
41    Topic20    8.695063      learn  303.378479   0.4393  -4.2620
406   Topic20    4.327501    learner   63.90